In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Thư viện
import pandas as pd
from scipy import stats
import numpy as np

## **B0 : KHÁM PHÁ DỮ LIỆU**





In [4]:
file_path = '/content/drive/MyDrive/DACN/DATA/housing.csv'
df = pd.read_csv(file_path)

In [ ]:
# Hiển thị dữ liệu 15 dòng đầu
df.head(15)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [5]:
# Thông tin về dữ liệu
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
# Kích thước về dữ liệu
df.shape

(20640, 10)

# **B1 : TIỀN XỬ LÝ**

In [7]:
# tính tỉ lệ khuyết
missing_value = (df.isnull().sum() / len(df))*100
# Convert về dạng DataFrame
missing_value_df = pd.DataFrame(missing_value, columns = ['tỉ lệ khuyết'])
#hiển thị tỉ lệ khuyết
missing_value_df

,tỉ lệ khuyết
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,1.002907
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [8]:
# Xử lý giá trị khuyết với cột total_bedrooms
median = df['total_bedrooms'].median()
df['total_bedrooms'] = df['total_bedrooms'].fillna(median)
# Kiểm tra lại giá trị khuyết
df_missing = pd.DataFrame(df.isnull().sum(),columns=['Giá trị khuyết'])
df_missing

,Giá trị khuyết
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0


In [9]:
# xử lý data trùng lặp
row_duplicated = df.duplicated().sum()
print('\nSố lượng data bị trùng : ',row_duplicated)
# Xóa Data trùng lặp
df = df.drop_duplicates()


Số lượng data bị trùng :  0


In [10]:
# Xử lý giá trị ngoại lai
def xu_ly_gia_tri_ngoai_lai(df, column):

  # Tính toán IQR (Interquartile Range)
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)
  IQR = Q3 - Q1

  # Xác định giới hạn trên và dưới để loại bỏ giá trị ngoại lai
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Thay thế các giá trị ngoại lai bằng giới hạn trên và dưới
  df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
  return df[column]

# Xử lý giá trị ngoại lai cho các cột liên quan đến gia nhà
columns_to_process = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population','households','median_income','median_house_value']

for column in columns_to_process:
  df[column] = xu_ly_gia_tri_ngoai_lai(df, column)

In [11]:
# Kiểm tra số nhãn trong dữ liệu dạng chữ
print('\nKiểm tra số nhãn')
print(df['ocean_proximity'], ' chứa ', len(df['ocean_proximity'].unique()), ' nhãn')
# Mã hóa các nhãn này thành dạng one-hot encoding
df_encode = pd.get_dummies(df['ocean_proximity']).astype(int)

# Kết hợp với dữ liệu gốc và loại bỏ cột ocean_proximity
df_new = pd.concat([df.drop(['ocean_proximity'], axis=1), df_encode], axis=1)


Kiểm tra số nhãn
0        NEAR BAY
1        NEAR BAY
2        NEAR BAY
3        NEAR BAY
4        NEAR BAY
           ...   
20635      INLAND
20636      INLAND
20637      INLAND
20638      INLAND
20639      INLAND
Name: ocean_proximity, Length: 20640, dtype: object  chứa  5  nhãn


In [12]:
# Hiển thị dữ liệu sau tiền xử lý (15 dòng đầu)
df_new.head(15)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.000,129.0,322.0,126.0,8.013025,452600.0,0,0,0,1,0
1,-122.22,37.86,21.0,5698.375,1106.0,2401.0,1092.5,8.013025,358500.0,0,0,0,1,0
2,-122.24,37.85,52.0,1467.000,190.0,496.0,177.0,7.257400,352100.0,0,0,0,1,0
3,-122.25,37.85,52.0,1274.000,235.0,558.0,219.0,5.643100,341300.0,0,0,0,1,0
4,-122.25,37.85,52.0,1627.000,280.0,565.0,259.0,3.846200,342200.0,0,0,0,1,0
5,-122.25,37.85,52.0,919.000,213.0,413.0,193.0,4.036800,269700.0,0,0,0,1,0
6,-122.25,37.84,52.0,2535.000,489.0,1094.0,514.0,3.659100,299200.0,0,0,0,1,0
7,-122.25,37.84,52.0,3104.000,687.0,1157.0,647.0,3.120000,241400.0,0,0,0,1,0
8,-122.26,37.84,42.0,2555.000,665.0,1206.0,595.0,2.080400,226700.0,0,0,0,1,0
9,-122.25,37.84,52.0,3549.000,707.0,1551.0,714.0,3.691200,261100.0,0,0,0,1,0


In [14]:
# Lưu dữ liệu vào tệp
df_new.to_csv('/content/drive/MyDrive/DACN/DATA/df_new_cleaned.csv', index=False)